케라스- 딥러닝 모델을 간편하게 만들고 훈련시킬 수 있는,
파이썬을 위한 딥러닝 프레임워크

케라스의 작업 흐름
1. 입력 텐서와 타깃 텐서로 이루어진 훈련 데이터 정의
2. 입력과 타깃을 매핑하는 층으로 이루어진 네트워크(또는 모델) 정의
3. 손실 함수, 옵티마이저, 모니터링하기 위한 측정 지표를 선택하여 학습 과정 설정
4. 훈련 데이터에 대해 모델의 fit() 메서드를 반복적으로 호출

In [1]:
#예시
from keras import layers

layer = layers.Dense(32, input_shape=(784,))#32개의 유닛으로 된 밀집 층

첫 번째 차원이 784인 2D 텐서만 입력으로 받는 층을 만들었다.
이 층은 첫 번째 차원 크기가 32로 변환된 텐서를 출력할 것이다.
그러므로 이 층의 하위 층은 32차원의 벡터를 입력으로 받아야 한다.

In [3]:
from keras import models

model = models.Sequential()
model.add(layers.Dense(32, activation='relu', input_shape=(784,)))
model.add(layers.Dense(10, activation='softmax'))

In [4]:
# 같은 모델을 함수형 API 사용하여 만들어 보기
input_tensor = layers.Input(shape=(784,))
x = layers.Dense(32, activation='relu')(input_tensor)
output_tensor = layers.Dense(10, activation='softmax')(x)

model = models.Model(inputs=input_tensor, outputs=output_tensor)

In [8]:
#컴파일 단계에서 학습 과정 설정
#여기에서 옵티마이저, 손실함수, 측정 지표 지정함
from tensorflow.keras import optimizers

model.compile(optimizer=optimizers.RMSprop(lr=0.001),
              loss='mse',
              metrics=['accuracy'])

In [ ]:
#입력 데이터의 넘파이 배열과 이에 상응하는 타깃 데이터를 
#모델의 fit() 메서드에 전달하면 학습 과정이 이루어진다.

#model.fit(input_tensor, target_tensor, batch_size=128, epochs=10)

In [28]:
#영화 리뷰 분류: 이진 분류 예제
#리뷰 텍스트를 기반으로 영화 리뷰를 긍정과 부정으로 분류

#데이터셋 불러오기
def load_data(path='../input/imdbdata/imdb.npz',
              num_words=None,
              skip_top=0,
              maxlen=None,
              seed=113,
              start_char=1,
              oov_char=2,
              index_from=3,
              **kwargs):
    
    path="../input/imdbdata/imdb.npz"
    with np.load(path, allow_pickle=True) as f:
        x_train, labels_train = f['x_train'], f['y_train']
        x_test, labels_test = f['x_test'], f['y_test']

    rng = np.random.RandomState(seed)
    indices = np.arange(len(x_train))
    rng.shuffle(indices)
    x_train = x_train[indices]
    labels_train = labels_train[indices]

    indices = np.arange(len(x_test))
    rng.shuffle(indices)
    x_test = x_test[indices]
    labels_test = labels_test[indices]

    if start_char is not None:
        x_train = [[start_char] + [w + index_from for w in x] for x in x_train]
        x_test = [[start_char] + [w + index_from for w in x] for x in x_test]
    elif index_from:
        x_train = [[w + index_from for w in x] for x in x_train]
        x_test = [[w + index_from for w in x] for x in x_test]

    if maxlen:
        x_train, labels_train = _remove_long_seq(maxlen, x_train, labels_train)
        x_test, labels_test = _remove_long_seq(maxlen, x_test, labels_test)
        if not x_train or not x_test:
            raise ValueError('After filtering for sequences shorter than maxlen=' +
                             str(maxlen) + ', no sequence was kept. '
                             'Increase maxlen.')

    xs = np.concatenate([x_train, x_test])
    labels = np.concatenate([labels_train, labels_test])

    if not num_words:
        num_words = max(max(x) for x in xs)

    if oov_char is not None:
        xs = [
            [w if (skip_top <= w < num_words) else oov_char for w in x] for x in xs
        ]
    else:
        xs = [[w for w in x if skip_top <= w < num_words] for x in xs]

    idx = len(x_train)
    x_train, y_train = np.array(xs[:idx],dtype=object), np.array(labels[:idx])
    x_test, y_test = np.array(xs[idx:],dtype=object), np.array(labels[idx:])

    return (x_train, y_train), (x_test, y_test)

  #훈련 데이터에서 제일 자주 나타나는 단어 1만 개만 사용
(train_data, train_labels), (test_data, test_labels) = load_data(num_words=10000)

In [37]:
#train_data와 test_data는 리뷰의 목록이다. 각 리뷰는 단어 인덱스의 리스트.
#train_labels와 test_labels는 부정을 나타내는 0과 긍정을 나타내는 1의 리스트.
train_labels

In [38]:
#가장 자주 등장하는 단어 1만개로 제한했기 때문에
#단어 인덱스는 10000을 넘지 않음
max([max(sequence) for sequence in train_data])

신경망에 숫자 리스트를 주입할 수는 없다. 리스트를 텐서로 바꿔야 한다.
여기서는 리스트를 원-핫 인코딩하여 0과 1의 벡터로 변환한다.

In [41]:
#정수 시퀀스를 이진 행렬로 인코딩
import numpy as np

def vectorize_sequences(sequences, dimension=10000):
    # 크기가 (len(sequences), dimension)이고 모든 원소가 0인 행렬을 만든다.
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1. # results[i]에서 특정 인덱스의 위치를 1로 만든다.
    return results

x_train = vectorize_sequences(train_data) # 훈련 데이터를 벡터로 변환
x_test = vectorize_sequences(test_data) # 테스트 데이터를 벡터로 변환

In [42]:
x_train[0]

In [43]:
y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')

입력 데이터= 벡터
레이블= 스칼라
Dense 층을 쌓을 때 결정해야 하는 두가지:
* 얼마나 많은 층을 사용할 것인가?
* 각 층에 얼마나 많은 은닉 유닛을 둘 것인가?

여기서는 16개의 은닉 유닛을 가진 2개의 층을 둘 것이며,
세 번째 층은 현재 리뷰의 감정을 스칼라 값의 예측으로 출력한다. 
은닉층의 활성화 함수로는 relu를 사용하고 마지막 층은 시그모이드를 사용한다.

In [44]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

손실 함수와 옵티마이저를 선택한다. 이진 분류 문제고 신경망의 출력이 확률이기 때문에 binary_crossentropy 손실이 적합하다. MSE도 사용할 수 있긴 하지만 확률을 출력하는 모델을
만들 땐 크로스엔트로피가 최선이라고 한다.

크로스엔트로피는 확률 분포 간의 차이를 측정한다. 여기서는 원본 분포와 예측 분포 사이를 측정한다.
옵티마이저는 rmsprop를 쓴다는데 뭔지모르겠다

In [45]:
#모델 컴파일
#케라스에 rmsprop, binary_crossentropy, accuracy가 포함되어 있어서
#문자열로 지정 가능함
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [47]:
#옵티마이저의 매개변수를 바꿔야 할 때 다음 코드를 사용
from tensorflow.keras import optimizers
model.compile(optimizer=optimizers.RMSprop(lr=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [48]:
#자신만의 손실 함수나 측정 함수를 전달해야 할 때 다음 코드를 사용
from keras import losses
from keras import metrics

model.compile(optimizer=optimizers.RMSprop(lr=0.001),
              loss=losses.binary_crossentropy,
              metrics=[metrics.binary_accuracy])

# 훈련 검증
모델 정확도를 측정하기 위해 원본 훈련 데이터에서 10000개의 셈플을 떼어
검증 세트를 만든다.

In [49]:
x_val = x_train[:10000]
partial_x_train = x_train[10000:]
y_val = y_train[:10000]
partial_y_train = y_train[10000:]

In [50]:
#모델에서 512개의 샘플씩 미니 배치를 만들어 20번의 에포크 동안 훈련
#(x_train과 y_train 텐서에 있는 모든 샘플에 대해 20번 반복)
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=20,
                    batch_size=512,
                    validation_data=(x_val, y_val))

In [51]:
history_dict = history.history
history_dict.keys()

In [52]:
#훈련과 검증 손실 그리기
import matplotlib.pyplot as plt

history_dict = history.history
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, 'bo', label='Training loss')  # ‘bo’는 파란색 점을 의미합니다.
plt.plot(epochs, val_loss, 'b', label='Validation loss') # ‘b’는 파란색 실선을 의미합니다.
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [53]:
# 훈련과 검증 정확도 그리기
plt.clf() # 그래프를 초기화합니다.
acc = history_dict['acc']
val_acc = history_dict['val_acc']

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

훈련 손실은 감소하고 훈련 정확도는 상승하는데, 검증 손실이랑 검증 정확도는 딱히,,,

 과대적합!
 
 그래프를 보니까 3번째 에포크에서부터 검증 정확도가 떨어지고 있다. 

In [56]:
#처음부터 다시 훈련하기. 근데 에포크를 4번까지만
model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

new = model.fit(x_train, y_train, epochs=4, batch_size=512)
results = model.evaluate(x_test, y_test)

In [55]:
results
#정확도가 훨씬 높아짐!!

In [58]:
model.predict(x_test)
#어떤 리뷰가 긍정일 확률
#어떤 샘플엔 확신(0.99나 0.01)을 가지고 있지만 어떤 샘플에 대해선 확신이 부족하다.(0.6 0.4...)

In [59]:
#은닉층의 갯수를 다르게 한다면 테스트 정확도가 높아질까?
#손실 함수를 다른 걸 써볼까?
#활성화 함수를 다른 걸 써볼까?